In [ ]:
# %load_ext autoreload
# %autoreload 2

import time
import os
import argparse
import cProfile
import pstats
import logging
from datetime import datetime

import numpy as np
import astropy.io.fits as pyfits
import corgihowfsc

from corgihowfsc.utils.howfsc_initialization import get_args, load_files
from corgihowfsc.sensing.GettingProbes import DefaultProbes

import eetc
from eetc.cgi_eetc import CGIEETC

import howfsc
from howfsc.control.cs import ControlStrategy
from howfsc.control.calcjtwj import JTWJMap

from howfsc.model.mode import CoronagraphMode

from howfsc.util.loadyaml import loadyaml
from howfsc.util.gitl_tools import param_order_to_list

from howfsc.gitl import howfsc_computation
from howfsc.precomp import howfsc_precomputation

from howfsc.scripts.gitlframes import sim_gitlframe

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')
defjacpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\data'
howfscpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\corgihowfsc' #os.path.dirname(os.path.abspath(corgihowfsc.__file__))


In [ ]:
# Load arguments
# defjacpath = None #os.path.join(os.path.dirname(howfscpath), 'jacdata')

precomp= 'load_all' if defjacpath is not None else 'precomp_all_once'

current_datetime = datetime.now()
folder_name = 'gitl_simulation_' + current_datetime.strftime("%Y-%m-%d_%H%M%S")
fits_name = 'final_frames.fits'
fileout_path = os.path.join(os.path.dirname(os.path.dirname(corgihowfsc.__file__)), 'data', folder_name, fits_name)

args = get_args(mode='nfov_band1', precomp=precomp, num_process=0, num_threads=1, fileout=fileout_path,jacpath=defjacpath)



In [ ]:
jacpath

In [ ]:
# Initialize variables etc

otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath

modelpath, cfgfile, jacfile, cstratfile, probefiles, hconffile, n2clistfiles = load_files(args, howfscpath)


# cfg
cfg = CoronagraphMode(cfgfile)

In [ ]:
for j in range(nlam):
    other[j] = dict()
    other[j]['lam'] = cfg.sl_list[j].lam
    other[j]['meas_intensity'] = nim

In [ ]:
# Initialize default probes class
probes = DefaultProbes('default')

# Get DM lists
# probefiles = {}
# probefiles[0] = probe0file
# probefiles[2] = probe1file
# probefiles[1] = probe2file
dm1_list, dm2_list, dmrel_list, dm10, dm20 = probes.get_dm_probes(cfg, probefiles, scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])

# Get probe amplitude lists (this step is slow)
plist, other = probes.get_probe_ap(cfg, dm1_list, dm2_list, other=otherlist)

In [ ]:
from corgihowfsc.utils.cgisim_gitl_frames import gen_cgisim_excam_frame_for_gitl


In [ ]:
exptime = 30
gain = 5000
dm1v = dm1_list[1]
dm2v = dm2_list[1]
cor = 'hlc'
bandpass = '1a'
crop = (0, 0, 500, 500)
frame = gen_cgisim_excam_frame_for_gitl(
        exptime, gain,
        dm1v, dm2v,
        cor, bandpass,
        crop,
        param_dict={},
        star_spectrum='a5v', vmag=2.56,  # del Leo
        polaxis=10,
        cleanrow=1024, cleancol=1024,
        )

In [ ]:
from corgihowfsc.utils.corgisim_gitl_frames import GitlImage


In [ ]:
imager = GitlImage()

exptime = 0.1
gain = 1
dm1v = dm1_list[1]
dm2v = dm2_list[1]
cor = 'hlc'
bandpass = '1a'
crop = (0, 0, 500, 500)

frame = imager.gen_corgisim_excam_frame_for_gitl(
            exptime, gain,
            dm1v, dm2v,
            cor, bandpass,
            crop,
            param_dict={},
            ccd=None,
            # star_spectrum='a5v', vmag=2.56,  # del Leo
            polaxis=10,
            cleanrow=1024, cleancol=1024,
    )

In [ ]:
plt.imshow(frame)

In [ ]:
estimate_data = DefaultEstimateData()
for i in range(niter):
    
    # Update parameters like the scale factor etc
    abs_dm1, abs_dm2, scale_factor_list, gain_list, exptime_list, \
            nframes_list, prev_c, next_c, next_time, status, other = \
            howfsc_computation(framelist, dm1_list, dm2_list, cfg, jac, jtwj_map,
                               croplist, prev_exptime_list,
                               cstrat, n2clist, hconf, iteration)
    
    
    otherlist.append(other)
    abs_dm1list.append(abs_dm1)
    abs_dm2list.append(abs_dm2)
    framelistlist.append(framelist)
    scalelistout.append(scale_factor_list)
    camlist.append([gain_list, exptime_list, nframes_list])

    print('-----------------------------------')
    print('Iteration: ' + str(iteration))
    print('HOWFSC computation time: ' + str(t1-t0))
    print('Previous contrast: ' + str(prev_c))
    print('Next contrast: ' + str(next_c))
    print('scales: ' + str(scale_factor_list))


    # new dm1_list, dm2_list
    dm1_list = []
    dm2_list = []
    for index in range(nlam):
        # DM1 same per wavelength
        dm1_list.append(abs_dm1)
        dm1_list.append(abs_dm1 + scale_factor_list[0]*dmrel_list[0])
        dm1_list.append(abs_dm1 + scale_factor_list[3]*dmrel_list[0])
        dm1_list.append(abs_dm1 + scale_factor_list[1]*dmrel_list[1])
        dm1_list.append(abs_dm1 + scale_factor_list[4]*dmrel_list[1])
        dm1_list.append(abs_dm1 + scale_factor_list[2]*dmrel_list[2])
        dm1_list.append(abs_dm1 + scale_factor_list[5]*dmrel_list[2])
        for j in range(ndm):
            # DM2 always same
            dm2_list.append(abs_dm2)
            pass
            pass

    framelist = estimate_data.get_estimate_data(cfg, dm1_list, dm2_list)

In [ ]:
cfg.sl_list[0]

In [ ]:
args.mode